In [24]:
import pandas as pd

In [25]:
df = pd.read_csv("spotify_millsongdata.csv")

In [26]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [27]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [28]:
df.shape

(57650, 4)

In [29]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [30]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [31]:
df.head(10)

,artist,song,text
0,Veruca Salt,Sleeping Where I Want,"This is my favorite disease, \r\nI am sleepin..."
1,Queens Of The Stone Age,How To Handle A Rope,Too late to think or filter anymore \r\nA bit...
2,Avril Lavigne,The Best Damn Thing,"Let me hear you say: ""hey, hey, hey"" (Hey, hey..."
3,Zebra,Wait Until The Summer's Gone,The time is right but don't be critical \r\nT...
4,Tina Turner,Addicted To Love,The lights are on but you're not home \r\nYou...
5,Religious Music,Carol Of The Bells,"Hark how the bells, \r\nSweet silver bells, ..."
6,Happy Mondays,Tart Tart,When he came out to the lock-up \r\nHe said I...
7,Savage Garden,You Can Still Be Free,Cool breeze and autumn leaves \r\nSlow motion...
8,The Monkees,You Told Me,"You told me you'd always stay, you told me! \..."
9,Proclaimers,Dance With Me,Music sounds through the room \r\nAnd you tak...


In [32]:
df['text'][0]

"This is my favorite disease,  \r\nI am sleeping in his dreams  \r\nAnd the TV is on  \r\nIt is seeping into me  \r\nIt is guiding my dreams  \r\nIt is only three  \r\nSo it's jeopardy  \r\nYou can go and burst my tiny bubble  \r\nBlow me off without a care  \r\nBut I am sleeping where I want to.  \r\nThis is my favorite time  \r\nI can sleep until the phone rings  \r\nAnd I doubt that it will.  \r\nYou can go and burst my tiny bubble  \r\nBlow me off without a care  \r\nBut I am sleeping where I want to  \r\n\r\n"

In [33]:
# df = df.sample(5000)

In [34]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [35]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [36]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [37]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [38]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Mohit
[nltk_data]     Vashist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [41]:
similarity[0]

array([1.        , 0.00714823, 0.01473784, ..., 0.0090633 , 0.03045272,
       0.01389485])

In [42]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [43]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [44]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))